In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

In [41]:
FullLabelPath = 'parelsnoer/parelsnoer_Labels_extensive3_pruned_alternative_binning.txt'
FullLabelFile = pd.read_csv(FullLabelPath,names=['ID','label','extra'],sep='\t',low_memory=False)

In [42]:
FullLabelFile

,ID,label,extra
0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0
1,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0
2,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0
3,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0
4,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0
...,...,...,...
11140,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0
11141,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0
11142,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0
11143,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0


In [43]:
FullLabelFile['label'].value_counts()

label
0    7305
3    2520
1     990
8     195
5      75
4      30
2      30
Name: count, dtype: int64

In [44]:
FullLabelFile.loc[FullLabelFile['label']==5,'label']=2

In [45]:
amountsPerClass = [1, 5, 10, 20]

In [46]:
def extractNiftiFilepathAndSlicenum(df):
    ID = df['ID']
    split = ID.rsplit('__s', 1)
    NiftiPath = split[0]
    NiftiPath = NiftiPath.replace('NIFTI_SLICES', 'NIFTI') + '.nii.gz'
    slicenum = int(split[1].split('.nii.gz')[0])
    return pd.Series({'NiftiPath': NiftiPath, 'slicenum': slicenum})

In [47]:
FilepathsAndSlicenums = FullLabelFile.apply(extractNiftiFilepathAndSlicenum, axis=1)

In [48]:
FullLabelFile = FullLabelFile.merge(FilepathsAndSlicenums, left_index=True, right_index=True)

In [49]:
FullLabelFile

,ID,label,extra,NiftiPath,slicenum
0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,7
1,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,5
2,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,8
3,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,13
4,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,9
...,...,...,...,...,...
11140,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,5
11141,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,7
11142,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,17
11143,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,10


In [50]:
for numTrainSamples in amountsPerClass:
    extension = os.path.splitext(FullLabelPath)
    basename = extension[0]
    extension = extension[1]
    df_train = pd.DataFrame()
    df_test = pd.DataFrame()
    for label in FullLabelFile['label'].unique():
        outname_train = f'{basename}_s{numTrainSamples}_train{extension}'
        outname_test = f'{basename}_s{numTrainSamples}_test{extension}'
        LabelFrame = FullLabelFile[FullLabelFile['label']==label]
        NiftiNames = LabelFrame['NiftiPath'].unique()
        try:
            train, test = train_test_split(NiftiNames, train_size=numTrainSamples, shuffle=True, random_state=42)
        except:
            train = NiftiNames
            test = []
        for tr in train:
            if df_train.empty:
                df_train = LabelFrame[LabelFrame['NiftiPath']==tr]
            else:
                temp = LabelFrame[LabelFrame['NiftiPath']==tr]
                df_train = pd.concat([df_train, temp],axis=0)
        for te in test:
            if df_test.empty:
                df_test = LabelFrame[LabelFrame['NiftiPath']==te]
            else:
                temp = LabelFrame[LabelFrame['NiftiPath']==te]
                df_test = pd.concat([df_test, temp],axis=0)
        df_train[['ID','label','extra']].to_csv(outname_train, index=False, header=False, sep='\t')
        df_test[['ID','label','extra']].to_csv(outname_test, index=False, header=False, sep='\t')

In [51]:
df_train

,ID,label,extra,NiftiPath,slicenum
3030,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,13
3031,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,14
3032,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,8
3033,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,19
3034,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,18
...,...,...,...,...,...
520,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,4,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,12
521,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,4,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,7
522,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,4,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,6
523,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,4,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,15


In [52]:
df_test

,ID,label,extra,NiftiPath,slicenum
1680,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,11
1681,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,16
1682,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,7
1683,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,15
1684,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,0,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,9
...,...,...,...,...,...
8425,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,1,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,6
8426,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,1,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,14
8427,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,1,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,5
8428,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,1,0,/trinity/home/r098375/DDS/data/parelsnoer/NIFT...,8


In [53]:
control = 'parelsnoer/parelsnoer_Labels_extensive3_pruned_alternative_binning_s10_train.txt'
ControlLabelFile = pd.read_csv(control,names=['ID','label','extra'],sep='\t',low_memory=False)

In [54]:
ControlLabelFile['label'].value_counts()

label
0    150
3    150
8    150
1    150
2    105
4     30
Name: count, dtype: int64